# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [ ]:
import pandas as pd
import requests

sinasc = pd.read_csv('SINASC_RO_2019.csv') # carregando a Base de Dados e transferindo-a para um DataFrame
print(sinasc.shape) # exibindo as dimensões da Base

# simultaneamente retirando as duplicatas e verificando se houve remoção quando comparado a original
sinasc.drop_duplicates().shape

In [ ]:
# como há 69 variáveis, apenas as 5 primeiras e 5 últimas estão exibidas, mas todas estão contabilizadas
sinasc.isna().sum()

In [ ]:
sinasc_filtrado = sinasc.loc[:, ["LOCNASC",
                                 "IDADEMAE",
                                 "ESTCIVMAE",
                                 "ESCMAE",
                                 "QTDFILVIVO",
                                 "GESTACAO",
                                 "GRAVIDEZ",
                                 "CONSULTAS",
                                 "APGAR5"]]
print(sinasc_filtrado.shape) # conferindo as dimensões da Base filtrada

sinasc_filtrado.isna().sum()

In [ ]:
sinasc_principal = sinasc_filtrado.dropna(subset= ["APGAR5"])

# 27028 -26925 =103, confirmando o dado anterior. Logo, foram retiradas 103 linhas (apenas 0,38% da Base de Dados)
print(sinasc_principal.shape)
sinasc_principal.isna().sum() # verificando a contagem de missings APGAR5 igual a 0

In [ ]:
# passando o valor 9 para todos os missing das variaveis
sinasc_principal["ESTCIVMAE"].fillna(9)
sinasc_principal["CONSULTAS"].fillna(9)

sinasc_principal.shape # conferindo a Base de Dados principal

In [ ]:
sinasc_principal["QTDFILVIVO"].fillna(0) # passando o valor 0 para todos os missings da variável

sinasc_principal.shape # conferindo a Base de Dados principal

In [ ]:
# bloco de análise para decisão coerente com as informações contidas na Base de Dados para as variáveis numéricas
print(sinasc_principal["LOCNASC"].mean())
print(sinasc_principal["LOCNASC"].median())
print(sinasc_principal["IDADEMAE"].mean())
print(sinasc_principal["IDADEMAE"].median())

In [ ]:
# bloco de análise para decisão coerente com as informações contidas na Base de Dados para as variáveis categóricas
print(pd.get_dummies(sinasc_principal["ESCMAE"]).sum())
print(pd.get_dummies(sinasc_principal["GESTACAO"]).sum())
print(pd.get_dummies(sinasc_principal["GRAVIDEZ"]).sum())

In [ ]:
# atribuição dos valores vazios aos mais adequados para cada variável
sinasc_principal["LOCNASC"].fillna(sinasc_principal["LOCNASC"].median())
sinasc_principal["IDADEMAE"].fillna(sinasc_principal["IDADEMAE"].median())
sinasc_principal["ESCMAE"].fillna("8 a 11 anos")
sinasc_principal["GESTACAO"].fillna("37 a 41 semanas")
sinasc_principal["GRAVIDEZ"].fillna("Única")

print(sinasc_principal.columns)

In [ ]:
# categorização dos índices em uma nova coluna
sinasc_principal.loc[sinasc_principal["APGAR5"] >= 8, "APGAR5_cat"] = "normal"
sinasc_principal.loc[(sinasc_principal["APGAR5"] > 5) & (sinasc_principal["APGAR5"] < 8), "APGAR5_cat"] = "asfixia leve"
sinasc_principal.loc[(sinasc_principal["APGAR5"] > 3) & (sinasc_principal["APGAR5"] <= 5), "APGAR5_cat"] = "asfixia moderada"
sinasc_principal.loc[sinasc_principal["APGAR5"] <= 3, "APGAR5_cat"] = "asfixia severa"

sinasc_principal.head(5)

In [ ]:
# como eu desejei renomear as colunas de acordo com a minha preferência de identificação, não houve necessidade do inplace
sinasc_principal.columns = ["local_nasc",
                            "idade_mae",
                            "estado_civ_mae",
                            "escol_mae",
                            "qtd_filho_vivo",
                            "tempo_gestacao",
                            "tipo_gravidez",
                            "qtd_consultas",
                            "valor_apgar5",
                            "cat_apgar5"]

sinasc_principal